In [ ]:
# default_exp distributed

In [ ]:
from fastai.test_utils import *
from fastai.callback.training import *
from fastai.callback.fp16 import *
from fastai.basics import *
from fastai.distributed import *

In [5]:
#hide
#cuda
#Tests that afer n_batches, parameters should all be the same in fp16 and fp32
class GetParameters(Callback):
    "Record non-overflowing parameters, and batches that overflowed in fp16"
    def __init__(self):
        self.overflows=[]
        self.ps=L()
    def after_backward(self):
        if grad_overflow([self.model.parameters()]): 
            self.overflows+=[int((self.iter))]
        self.ps+=to_detach(L([p.clone() for p in self.model.parameters()]))
#Used to skip batches in fp32, which overflow in fp16
class Skip(Callback):
    run_after=GetParameters
    "Skips and zero_grads specified batches"
    def __init__(self,skips):
        self.skips=skips
    def after_backward(self):
        if self.iter in self.skips: 
            self.learn.opt.zero_grad()
            raise CancelBatchException()
n_batches=10
bs=8
seed=random.randint(0,2**32-1)
with no_random(seed): 
    #to_native_fp16 will not convert default synth_learner model
    model=nn.Sequential(nn.Linear(1,1))
    db=synth_dbunch(bs=bs,n_train=n_batches,n_valid=n_batches,cuda=True)
    learn = synth_learner(model=model,data=db,cbs=[GetParameters()])
    #learn.to_native_fp16()
    with learn.distrib_ctx(1):
        print(learn.cbs)
        learn.fit(1, lr=0.01)
pa=learn.get_parameters.ps
overflows=learn.get_parameters.overflows
with no_random(seed): 
    model=nn.Sequential(nn.Linear(1,1))
    db=synth_dbunch(bs=bs,n_train=n_batches,n_valid=n_batches,cuda=True)
    learn = synth_learner(model=model,data=db,cbs=[Skip(overflows),GetParameters()])
    learn.fit(1, lr=0.01)
pb=learn.get_parameters.ps
#test all batches had same parameters in fp16 and fp32
test_close(pa,pb,eps=1e-3) 

[TrainEvalCallback, Recorder, ProgressCallback, GetParameters]


epoch,train_loss,valid_loss,time
0,7.008786,1.418512,00:00


epoch,train_loss,valid_loss,time
0,7.008786,1.418512,00:00
